# Quadruped Navigation/Motion Control with Text Commands


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pybullet as p
import pybullet_data
import time
import math

from scripts.agent import AgentSAC
from scripts.utils import get_state, get_spawn, get_goal, get_reward_done

In [3]:
# DEFINE VARIABLES AND HYPERPARAMETERS
PI = math.pi
NUM_EPISODES = 500
ACTION_DURATION = 0.2
MAX_EPISODE_TIME = 30
MAX_STEPS_PER_EPISODE = int(MAX_EPISODE_TIME / ACTION_DURATION)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create Agent

In [4]:
agent = AgentSAC(state_dim=386, action_dim=12).to(device)

Start Pybullet Client


In [5]:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
for episode in range(NUM_EPISODES):
    # Reset simulation and upload models
    p.resetSimulation()
    p.setGravity(0, 0, -9.8)    # set gravity
    
    # Load Plane model
    planeId = p.loadURDF('plane.urdf')

    # Load Robot Model
    init_x, init_y, init_z_orientation = get_spawn()
    init_pos = [init_x, init_y, 0.48] 
    init_orientation = p.getQuaternionFromEuler([0, 0, init_z_orientation])
    robotId = p.loadURDF('quadruped/quadruped.urdf', basePosition=init_pos, baseOrientation=init_orientation)
    print(f'placed robot in: {init_x}, {init_y}')
    num_joints = p.getNumJoints(robotId) - 1 # One less for LiDAR
    
    # Get LiDAR idx
    lidar_idx = p.getNumJoints(robotId) - 1
    
    # get goal
    goal = get_goal(init_x, init_y)
    print(f'goal set to: {goal[0], goal[1]}')
    time.sleep(1.0)
    
    # get state
    state = get_state(robotId, goal)
    
    total_reward = 0
    done = False
    for step in range(MAX_STEPS_PER_EPISODE):

        # Select and apply action
        action, log_prob, _ = agent.Actor.sample(state)
        action_ = action.detach().cpu().numpy()
        p.setJointMotorControlArray(robotId, range(num_joints), controlMode=p.POSITION_CONTROL, targetPositions=action_)
        
        # Step through the simulation
        start_time = time.time()
        while time.time() - start_time < ACTION_DURATION:
            p.stepSimulation()
            time.sleep(1/240.0)
        
        # Get next state
        next_state = get_state(robotId, goal)
        
        # Calculate reward and done
        reward, done = get_reward_done(robotId, goal)
        
        # Store transition into Replay Buffer
        agent.buffer.push(state, action, next_state, reward, done)
        
        # Optimize networks
        if len(agent.buffer.memory) >= agent.batch_size:
            states, actions, rewards, next_states, dones = agent.buffer.sample(agent.batch_size)
            
            # Optimize Critic
            agent.optimize_critic(states, actions, rewards, next_states, dones)
            
            # Optimize Actor
            agent.optimize_actor(states)
            
            # Update Target Critic
            agent.update_target()
                        
        
        # Transition into next state
        state = next_state
        total_reward += reward
        
        # Break if done
        if done:
            break
        
        